In [67]:
import os
%pwd

'c:\\Users\\akash\\Documents\\Workspace\\Scaler\\Projects\\Text-Summarizer-NLP'

In [29]:
os.chdir("../")
%pwd

'c:\\Users\\akash\\Documents\\Workspace\\Scaler\\Projects\\Text-Summarizer-NLP'

In [45]:
#updating entity

from dataclasses import dataclass
from pathlib import Path
#defining the return type of a function
#this is the entity

#3. Update entity

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path  


In [46]:
%pwd

'c:\\Users\\akash\\Documents\\Workspace\\Scaler\\Projects\\Text-Summarizer-NLP'

In [42]:
from textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
print(PARAMS_FILE_PATH)
CONFIG_FILE_PATH

params.yaml


WindowsPath('config/config.yaml')

In [47]:
#4. update the configuration manager in src config


from textSummarizer.constants import * 
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH) -> None:
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

        

In [48]:
# 5. update the components
import os
#to download the data
import urllib.request as request
#to unzip the data
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [65]:
filename,headers=request.urlretrieve(
                url="https://github.com/jakashcode/Text-Summarizer-NLP/blob/main/summarizer-data.zip",
                filename='artifacts/data_ingestion/data.zip'
)
print(filename)
print(headers)

artifacts/data_ingestion/data.zip
Server: GitHub.com
Date: Tue, 14 Nov 2023 18:33:21 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"2f18745a933ae9dd97446d2f7d67cc33"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github.com/assets-cdn/worker/ gist.github.com/assets-cdn/worker/; connect-src 'self' uploads.github.com www.githubstatus.com collector.github.com raw.githubusercontent.com api.githubcopilot.com api.github.com github-cloud.s3.amazonaws.com github-production-repository-file-5c1aeb.s3.amazonaws.com github-production-upload-manifest-file-7fdce7.s3.amazonaws.com github-production-user-asset-6210df.s3.amazonaws.com cdn

In [51]:
class DataIngestion:
    def __init__(self,config=DataIngestionConfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            print("*"*20)
            print(self.config.source_URL)
            print(self.config.local_data_file)
            print('#'*20)
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file)
            
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [68]:

#creating the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    #data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-15 00:05:49,837: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-15 00:05:49,840: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-15 00:05:49,842: INFO: common: created directory at: artifacts]
[2023-11-15 00:05:49,843: INFO: common: created directory at: artifacts/data_ingestion]
